In [1]:
import keras
import firebase_admin
from firebase_admin import ml
from firebase_admin import credentials

firebase_admin.initialize_app(
  credentials.Certificate('/home/cam/Documents/heartfelt_firebase_dev/.env'),
  options={
      'storageBucket': 'heartfelt-0.appspot.com',
  })

# Load a Keras model, convert it to TensorFlow Lite, and upload it to Cloud Storage
model = keras.models.load_model('/home/cam/Documents/Enceladus/notebooks/artifacts/model-vital-bee-206:v19')
source = ml.TFLiteGCSModelSource.from_keras_model(model)

model = ml.Model(
    display_name="enceladus",   # This is the name you use from your app to load the model.
    tags=["v1"],             # Optional tags for easier management.
    model_format=source)

# Add the model to your Firebase project and publish it
new_model = ml.create_model(model)
ml.publish_model(new_model.model_id)

2022-12-15 15:54:18.893919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-15 15:54:18.945275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-15 15:54:18.945544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-15 15:54:18.946628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

INFO:tensorflow:Assets written to: /tmp/tmpw8a5fp07/assets


INFO:tensorflow:Assets written to: /tmp/tmpw8a5fp07/assets
2022-12-15 15:54:40.272444: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-15 15:54:40.272463: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-15 15:54:40.272906: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpw8a5fp07
2022-12-15 15:54:40.291433: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-12-15 15:54:40.291452: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpw8a5fp07
2022-12-15 15:54:40.350975: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-15 15:54:40.373877: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-12-15 15:54:40.733945: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

TypeError: Model format must be a ModelFormat object.

In [ ]:
from Enceladus.jobs import TestingPipeline

TestingPipeline(
    data_path='/home/cam/Documents/database_tools/data-2022-11-08/mimic3/',
    scaler_path='/home/cam/Documents/database_tools/data-2022-11-08/mimic3/scalers_MinMax.pkl',
    scaler_type='MinMax',
    model_path='heartfelt/Enceladus/model-vital-bee-206:v19',
).run()

In [2]:
from Enceladus.jobs import TestingPipeline

TestingPipeline(
    data_path='/home/cam/Documents/database_tools/vitaldb/',
    scaler_path='/home/cam/Documents/database_tools/vitaldb/scalers_1670914756482882232.pkl',
    scaler_type='MinMax',
    model_path='heartfelt/Enceladus/model-vital-bee-206:v19',
).run()

Loading data...
Reading test split.
test


1193it [00:00, 6813.76it/s]


wandb: Downloading large artifact model-vital-bee-206:v19, 58.89MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.0


Generating predictions...
38/38 [==============================] - 1s 7ms/step
Rescaling data...
Calculating error...


100%|██████████| 1193/1193 [00:00<00:00, 2968.05it/s]

Systolic BP
37.7% < 15mmHg
23.3% < 10mmHg
12.5% < 5mmHg

Diastolic BP
67.5% < 15mmHg
49.0% < 10mmHg
28.2% < 5mmHg



In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl

with open('model-vital-bee-206:v19_error.pkl', 'rb') as f:
    data = pkl.load(f)

np.mean(data['sbp_abs_err'])

In [ ]:
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=data['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=data['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

fig

In [ ]:
np.mean(data['dbp_abs_err'])

In [3]:
import numpy as np
import pandas as pd
import pickle as pkl
from plotly.subplots import make_subplots

with open('model-vital-bee-206:v19_predictions.pkl', 'rb') as f:
    data = pkl.load(f)
    
ppg, vpg, apg, abp, pred = data.values()

In [34]:
i = 18

In [35]:
fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig